In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array
from QQuantLib.AA.amplitude_amplification import grover
from QQuantLib.utils.utils import get_histogram
class Zalo:
    
    def __init__(self, p_X, f_X):
        
        #Test condition in p(x)
        print('p(x) condition: {}'.format(np.sum(p_X) == 1))
        print('f(x) condition: {}'.format(np.max(f_X) <= 1))
        
        self.p_gate = load_probability(p_X)
        self.f_gate = load_array(np.sqrt(f_X))
        
        self.oracle = qlm.QRoutine()
        register = self.oracle.new_wires(self.f_gate.arity)
        self.oracle.apply(self.p_gate, register[:self.p_gate.arity])
        self.oracle.apply(self.f_gate, register)
        
        self.target = [0]
        self.index = [self.oracle.arity-1]
        
    def iqpe(self, n_cbits, shots, linalg_qpu):
        from QQuantLib.PE.iterative_quantum_pe import IterativeQuantumPE
        self.q_gate = grover(self.oracle, self.target, self.index)
        
        iqpe_dict = {
            'initial_state': self.oracle,
            'unitary_operator': self.q_gate,
            'qpu' : linalg_qpu,
            'cbits_number' : n_cbits,
            'shots': 100
        }
        
        self.pe_iqft = IterativeQuantumPE(**iqpe_dict)
        self.iqpe.iqpe()
        pdf = self.IQPE.sumary
        pdf['P']=np.cos(pdf['theta_90'])**2
        
    def qpe(self, n_cbits, shots, linalg_qpu):
        from QQuantLib.PE.phase_estimation_wqft import PhaseEstimationwQFT
        self.q_gate = grover(self.oracle, self.target, self.index)
        qpe_dict = {
            'initial_state': self.oracle,
            'unitary_operator': self.q_gate,
            'qpu' : linalg_qpu,
            'cbits_number' : n_cbits,
            'shots': 100
        }
        self.pe_iqft = PhaseEstimationwQFT(**qpe_dict)     
        self.pe_iqft.pe_wqft()
        
    def mlae(self, schedule, linalg_qpu):
        
        from  QQuantLib.AE.maximum_likelihood_ae  import MLAE
        
        target = [0]
        index = [self.oracle.arity-1]
        
        self.mlae = MLAE(
            self.oracle,
            target = self.target,
            index = self.index,
            schedule = schedule)
        angle = self.mlae.optimize()
        
    
    def iqae(self, linalg_qpu, epsilon = 0.001, alpha = 0.05, N = 1000):
        from QQuantLib.AE.iterative_quantum_ae import IQAE
        self.iqae = self.IQAE(oracle = self.oracle1,target = self.target, index = self.index)
        
        epsilon = 0.001
        N = 1000
        alpha = 0.05         
        bounds = self.iqae.run(epsilon = epsilon,N = N,alpha = alpha)
        


In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 4

In [ ]:
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
#Test condition in p(x)
print('p(x) condition: {}'.format(np.sum(p_X) == 1))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

In [ ]:
gon = Zalo(p_X, f_X)

In [ ]:
gon